In [1]:
from comparison_algorithms.pytket_dqc.distributors.partitioning_heterogeneous import (
    PartitioningAnnealing,
    PartitioningHeterogeneous,
    PartitioningHeterogeneousEmbedding
)
from pytket import Circuit, OpType
from comparison_algorithms.pytket_dqc.networks.nisq_network import NISQNetwork
from comparison_algorithms.pytket_dqc.utils.gateset import DQCPass
from pytket import Circuit, OpType
from comparison_algorithms.pytket_dqc.placement.placement import Placement
from comparison_algorithms.pytket_dqc.distributors.distributor import Distribution
from comparison_algorithms.pytket_dqc.distributors.cover_embedding import CoverEmbedding, CoverEmbeddingSteiner, CoverEmbeddingSteinerDetached
from comparison_algorithms.pytket_dqc.circuits.hypergraph_circuit import HypergraphCircuit
from pytket.circuit.display import render_circuit_jupyter


network = NISQNetwork([[0, 1], [1, 3], [0,2], [2, 3]], {0: [0, 1], 1: [2,3], 2: [4, 5], 3: [6, 7]})
circ = Circuit(7).CX(0, 1).CX(1,2).CX(1, 3).CX(3, 4).CX(3, 5).CX(1, 6)
# render_circuit_jupyter(circ)
DQCPass().apply(circ)
# render_circuit_jupyter(circ)
placement = PartitioningAnnealing().distribute(circ, network).placement.to_dict()
sorted_dict = {k: placement[k] for k in sorted(placement, reverse=False)}
for k, v in sorted_dict.items():
    print(f"Qubit {k}: {v}")
new_placement = {k : v for k, v in placement.items() if k < 7}

Qubit 0: 1
Qubit 1: 1
Qubit 2: 0
Qubit 3: 3
Qubit 4: 2
Qubit 5: 3
Qubit 6: 0
Qubit 7: 1
Qubit 8: 0
Qubit 9: 1
Qubit 10: 0
Qubit 11: 2
Qubit 12: 3


In [3]:
placement = Placement(placement)
# placement = Placement({0: 1, 1: 2, 2: 0, 3: 0, 4: 0})
distribution = Distribution(HypergraphCircuit(circ), new_placement, network)
assert distribution.is_valid()

circ_with_dist = distribution.to_pytket_circuit()


AttributeError: 'dict' object has no attribute 'is_valid'

In [40]:
distribution.get_qubit_mapping()

{q[0]: server_1[0],
 q[1]: server_1[1],
 q[2]: server_0[0],
 q[3]: server_2[0],
 q[4]: server_3[0],
 q[5]: server_2[1],
 q[6]: server_0[1]}

In [42]:
distribution.cost()

3

In [41]:
render_circuit_jupyter(circ_with_dist)

In [145]:
from pytket.qasm import circuit_to_qasm_str
from pathlib import Path

qasm_fp = Path("/home/lty/workbench/quantum_compilation/quantum-chip-network/src/comparison_algorithms/pytket_dqc/test.qasm")
print(circuit_to_qasm_str(circ_with_dist))

OPENQASM 2.0;
include "qelib1.inc";

qreg server_0[2];
qreg server_1[2];
qreg server_1_link_register[1];
qreg server_2[1];
qreg server_2_link_register[1];
rz(1.0*pi) server_0[0];
rz(1.0*pi) server_0[1];
rz(3.0*pi) server_1[0];
rz(1.0*pi) server_1[1];
rz(1.0*pi) server_2[0];
h server_0[0];
h server_0[1];
h server_1[1];
h server_2[0];
barrier server_0[0],server_1_link_register[0];
cu1(1.0*pi) server_1[1],server_1[0];
h server_1[1];
rz(1.0*pi) server_1[1];
barrier server_1[1],server_2_link_register[0];
cu1(1.0*pi) server_1_link_register[0],server_1[1];
cu1(1.0*pi) server_2[0],server_2_link_register[0];
barrier server_1_link_register[0],server_0[0];
barrier server_2_link_register[0],server_1[1];
h server_2[0];
h server_0[0];
rz(3.0*pi) server_2[0];
cu1(1.0*pi) server_0[1],server_0[0];
h server_0[1];
rz(3.0*pi) server_0[1];



In [146]:
with open(qasm_fp, "w") as f:
    f.write(circuit_to_qasm_str(circ_with_dist))

In [140]:
render_circuit_jupyter(circ_with_dist)

In [141]:
distribution.cost()

2

In [ ]:
from comparison_algorithms.pytket_dqc.distributors.partitioning_heterogeneous import (
    PartitioningAnnealing,
    PartitioningHeterogeneous,
    PartitioningHeterogeneousEmbedding
)
from comparison_algorithms.pytket_dqc.networks.nisq_network import NISQNetwork
from comparison_algorithms.pytket_dqc.utils.gateset import DQCPass
from pytket import Circuit, OpType


network = NISQNetwork([[0,1], [0,2]], {0:[0], 1:[1], 2:[2]})
circ = Circuit(2)
circ.add_gate(OpType.CU1, 1.0, [0, 1]).H(0).Rz(0.3,0).H(0).add_gate(OpType.CU1, 1.0, [0, 1])
# circ = Circuit(7).CX(0,1).CX(1,2).CX(2,3).CX(3,4).CX(4,5).CX(5, 6)
# pass_circ = DQCPass().apply(circ)

distribution = PartitioningHeterogeneousEmbedding().distribute(circ, network)

: 

In [8]:
print(distribution.placement.to_dict())

{0: 0, 2: 0, 3: 0, 1: 0}


In [1]:

from pathlib import Path
from qiskit import QuantumCircuit
from comparison_algorithms.pytket_dqc.init_pytket_dqc_map import tket_dqc_map
from frontend.chips_info_reader import QuHardwareInfoReader
qasm_fp = Path(
    "/home/lty/workbench/quantum_compilation/quantum-chip-network/tests/benchmark/Bench/Dis_Bench/medium/qaoa_n150_p20/qaoa_n150_p20.qasm"
)
config_fp = Path(
    "/home/lty/workbench/quantum_compilation/quantum-chip-network/tests/hardware_info/medium_chip_net/medium_chip_net_starRing_two_rmCoupling_6mumbai.json"
)
qu_circ = QuantumCircuit.from_qasm_file(qasm_fp)
chips_net_obj = QuHardwareInfoReader(config_fp)
_, chips_network = chips_net_obj.get_hardware_info(chip_type="ibm")
time_cost, qubit_group_res, init_map = tket_dqc_map(qu_circ, chips_network)
print(qubit_group_res)

{0: [0, 1, 3, 5, 11, 22, 23, 31, 40, 55, 64, 77, 79, 83, 84, 90, 91, 107, 128, 136, 137, 144, 145, 148], 1: [6, 16, 19, 21, 24, 34, 35, 43, 45, 47, 58, 59, 62, 72, 74, 81, 93, 97, 98, 100, 103, 106, 113, 114, 121, 123, 131], 5: [4, 37, 42, 44, 50, 53, 60, 63, 65, 78, 86, 89, 99, 104, 108, 109, 112, 115, 120, 122, 124, 126, 127, 134, 140, 141, 142], 4: [7, 9, 13, 14, 20, 26, 29, 32, 36, 38, 39, 48, 54, 57, 82, 116, 117, 135, 143, 146, 149], 2: [2, 8, 10, 12, 27, 28, 30, 33, 46, 61, 66, 69, 70, 71, 73, 85, 87, 88, 92, 94, 96, 101, 111, 118, 125, 129, 130], 3: [15, 17, 18, 25, 41, 49, 51, 52, 56, 67, 68, 75, 76, 80, 95, 102, 105, 110, 119, 132, 133, 138, 139, 147]}


In [2]:
from comparison_algorithms.pytket_dqc.distributors.partitioning_heterogeneous import (
    PartitioningAnnealing,
)
from pytket import Circuit
from pytket.qasm import circuit_from_qasm
from pytket.extensions.qiskit import qiskit_to_tk
from qiskit import QuantumCircuit
from comparison_algorithms.pytket_dqc.networks.nisq_network import NISQNetwork
from comparison_algorithms.pytket_dqc.utils.gateset import DQCPass
from pathlib import Path
from global_config import repo_path, tests_path

from frontend.chips_info_reader import QuHardwareInfoReader
config_fp = Path("/home/lty/workbench/quantum_compilation/quantum-chip-network/tests/hardware_info/medium_chip_net/medium_chip_net_starRing_two_rmCoupling_6mumbai.json")
chips_net_obj = QuHardwareInfoReader(config_fp)
_, chips_network = chips_net_obj.get_hardware_info(chip_type="ibm")
chip_topology_graph = chips_network.obtain_chip_network()
server_coupling = []
edges = chip_topology_graph.edges()
for edge in edges:
    if [edge[0], edge[1]] not in server_coupling and [edge[1], edge[0]] not in server_coupling:
        server_coupling.append([edge[0], edge[1]])
server_qubits = chips_network.get_each_chip_qubits_idx()
network = NISQNetwork(server_coupling=server_coupling, server_qubits=server_qubits)

qasm_fp = Path("/home/lty/workbench/quantum_compilation/quantum-chip-network/tests/benchmark/Bench/medium/useful/cat_n130_transpiled.qasm")
qiskit_circ = QuantumCircuit.from_qasm_file(qasm_fp)
vq_idx_list = [q.index for q in qiskit_circ.qubits]
circ = qiskit_to_tk(qiskit_circ)
# circ = Circuit(3).CZ(0, 1).CZ(1, 2)
DQCPass().apply(circ)
# network = NISQNetwork(
#     server_coupling=[[0, 1], [1, 2]],
#     server_qubits={0: [0], 1: [1], 2: [2]},
# )

circ_partitioner = PartitioningAnnealing()
res = circ_partitioner.distribute(circ, network)

INFO:qiskit.compiler.assembler:Total Assembly Time - 0.21958 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.05174 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04292 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04172 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.03958 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04029 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.03934 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04101 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04387 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04053 (ms)
INFO:qiskit.compiler.assembler:Total Assembly Time - 0.04101 (ms)
/tmp/ipykernel_2994759/1681829850.py:28: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  vq_idx_list = [q.index for q in qiskit_circ.qubits]


In [36]:
num_vqubits = 100
chip_topology_graph = chips_network.obtain_chip_network()
# ep_capacity = chips_network.get_each_chip_capacity()
ep_capacity= {0:27, 1:27, 2: 27, 3: 28, 4: 29, 5:30}

sorted_ep_capacity = sorted(ep_capacity.items(), key=lambda x: x[1], reverse=True)
# print(sorted_ep_capacity)
add_sum_pqubits = sorted_ep_capacity[0][1]
used_nodes = [sorted_ep_capacity[0][0]]
while add_sum_pqubits < num_vqubits:
    neighbor_nodes = []

    for used_node in used_nodes:
        for neighbor_node in chip_topology_graph.neighbors(used_node):
            if neighbor_node not in used_nodes and neighbor_node not in neighbor_nodes:
                neighbor_nodes.append(neighbor_node)
    
    sorted_neighbor_nodes = sorted(neighbor_nodes, key=lambda node: ep_capacity[node], reverse=True)

    print("used nodes:",used_nodes)
    print("neighbors:",neighbor_nodes)

    add_sum_pqubits += ep_capacity[sorted_neighbor_nodes[0]]
    used_nodes.append(sorted_neighbor_nodes[0])
# print(used_nodes)

used nodes: [5]
neighbors: [0, 1, 2, 3, 4]
used nodes: [5, 4]
neighbors: [0, 1, 2, 3]
used nodes: [5, 4, 3]
neighbors: [0, 1, 2]


In [1]:
from pathlib import Path
from qiskit import QuantumCircuit
from frontend.chips_info_reader import QuHardwareInfoReader
from comparison_algorithms.pytket_dqc.init_pytket_dqc_map import tket_dqc_map
qasm_fp = Path("/home/lty/workbench/quantum_compilation/quantum-chip-network/tests/benchmark/Bench/medium/useful/cat_n130_transpiled.qasm")
config_fp = Path("/home/lty/workbench/quantum_compilation/quantum-chip-network/tests/hardware_info/medium_chip_net/medium_chip_net_starRing_two_rmCoupling_6mumbai.json")

qu_circ = QuantumCircuit.from_qasm_file(qasm_fp)
chips_net_obj = QuHardwareInfoReader(config_fp)
_, chips_network = chips_net_obj.get_hardware_info(chip_type="ibm")
qubit_group_res, init_map = tket_dqc_map(qu_circ, chips_network)

In [3]:
qubit_group_res

{5: [14,
  15,
  16,
  17,
  39,
  58,
  59,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  106,
  107,
  108,
  117,
  118,
  119,
  120,
  121,
  122,
  123],
 0: [25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  82,
  83,
  84,
  85,
  86,
  102,
  103,
  104,
  105,
  124,
  125,
  126,
  127,
  128,
  129],
 1: [18,
  19,
  20,
  21,
  22,
  23,
  24,
  35,
  36,
  37,
  38,
  54,
  55,
  56,
  57,
  78,
  79,
  80,
  81,
  87,
  88,
  89,
  90,
  91,
  92],
 2: [0,
  1,
  2,
  3,
  4,
  5,
  6,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  72,
  73,
  74,
  75,
  76,
  77],
 3: [7,
  8,
  9,
  10,
  11,
  12,
  13,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116]}

In [2]:

init_map

{Qubit(QuantumRegister(130, 'q'), 0): 54,
 Qubit(QuantumRegister(130, 'q'), 1): 55,
 Qubit(QuantumRegister(130, 'q'), 2): 56,
 Qubit(QuantumRegister(130, 'q'), 3): 57,
 Qubit(QuantumRegister(130, 'q'), 4): 58,
 Qubit(QuantumRegister(130, 'q'), 5): 59,
 Qubit(QuantumRegister(130, 'q'), 6): 60,
 Qubit(QuantumRegister(130, 'q'), 7): 81,
 Qubit(QuantumRegister(130, 'q'), 8): 82,
 Qubit(QuantumRegister(130, 'q'), 9): 83,
 Qubit(QuantumRegister(130, 'q'), 10): 84,
 Qubit(QuantumRegister(130, 'q'), 11): 85,
 Qubit(QuantumRegister(130, 'q'), 12): 86,
 Qubit(QuantumRegister(130, 'q'), 13): 87,
 Qubit(QuantumRegister(130, 'q'), 14): 135,
 Qubit(QuantumRegister(130, 'q'), 15): 136,
 Qubit(QuantumRegister(130, 'q'), 16): 137,
 Qubit(QuantumRegister(130, 'q'), 17): 138,
 Qubit(QuantumRegister(130, 'q'), 18): 27,
 Qubit(QuantumRegister(130, 'q'), 19): 28,
 Qubit(QuantumRegister(130, 'q'), 20): 29,
 Qubit(QuantumRegister(130, 'q'), 21): 30,
 Qubit(QuantumRegister(130, 'q'), 22): 31,
 Qubit(QuantumReg